In [3]:
from datetime import datetime
import logging
import sys
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score
import lightgbm as lgb
import optuna
from optuna.integration import LightGBMPruningCallback

import warnings
warnings.filterwarnings('ignore')

optuna_dir = "..//optuna"

test_df = pd.read_csv(r'..//data//test.csv')
train_df = pd.read_csv(r'..//data//train.csv')
train_extra_df = pd.read_csv(r'..//data//training_extra.csv')
train_df = pd.concat([train_df, train_extra_df], ignore_index=True)

def prepare_data(df: pd.DataFrame, is_train: bool = True):
    """
    Prepares the dataset for training or testing by renaming columns, handling missing values,
    converting categorical and numerical features, and creating new features.
    
    Args:
        df (pd.DataFrame): The input dataframe (train or test).
        is_train (bool): Indicates if the dataframe is training data (default is True).
        
    Returns:
        pd.DataFrame: The processed dataframe.
    """
    
    # Define the column names
    columns = [
        'id', 'brand', 'material', 'size', 'compartments', 
        'laptop_compartment', 'is_waterproof', 'style', 'color', 
        'weight_capacity'
    ]
    
    if is_train:
        columns.append('price')
    
    df.columns = columns
    df = df.drop(columns='id')
    
    # Define the mapping for Size conversion
    size_mapping = {"Small": 1, "Medium": 2, "Large": 3}
    df["size_int"] = df["size"].map(size_mapping).fillna(0).astype(int)
    
    # Handle weight capacity
    df['weight_capacity'] = df['weight_capacity'].fillna(0)
    df['weight_capacity_int'] = df['weight_capacity'].astype(int)
    df['weight_capacity_size'] = df['weight_capacity'] * df['size_int']
    
    # Convert categorical columns
    df['compartments'] = df['compartments'].astype('category')
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    df[cat_cols] = df[cat_cols].astype('category')
    
    # Convert boolean columns to integer type
    df['laptop_compartment'] = df['laptop_compartment'].cat.codes.fillna(-1).astype(int)
    df['is_waterproof'] = df['is_waterproof'].cat.codes.fillna(-1).astype(int)
    
    return df

# Apply function to train and test datasets
train_df = prepare_data(train_df, is_train=True)
test_df = prepare_data(test_df, is_train=False)

numeric_cols = ['weight_capacity', 'weight_capacity_int', 'weight_capacity_size', 'size_int', 'is_waterproof']
cat_cols = ['brand', 'material', 'compartments', 'color']
target = 'price'
model_features = [
    'weight_capacity', 'weight_capacity_int', 'weight_capacity_size', 'size_int', 'color', 'compartments', 'brand', 'material', 'is_waterproof'
]

X = train_df[model_features].copy()
y = train_df[target]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, .35),
        'learning_rate': trial.suggest_loguniform('learning_rate', .01, 0.05),
        'max_bin': trial.suggest_int('max_bin', 10, 255),
        # 'max_bin': trial.suggest_int('max_bin', 100000, 130000),
        'max_depth': 10,
        'min_child_samples': trial.suggest_int('min_child_samples', 40, 80),
        'min_split_gain': 0.5,
        'n_estimators': trial.suggest_int('n_estimators', 9000, 15000),
        'num_leaves': trial.suggest_int('num_leaves', 900, 1200),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.5, 2.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.000001, 0.00001),
        'subsample': trial.suggest_float('subsample', 0.2, 0.25),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
        'device': 'gpu'
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(
        params=param,
        train_set=train_data,
        num_boost_round=700,
        valid_sets=[train_data, valid_data],
        valid_names=['train_0', 'valid_0'],
        callbacks=[
            LightGBMPruningCallback(trial, "rmse", valid_name="valid_0"),
            lgb.log_evaluation(-1)                   # Suppress training logs
        ]
    )
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M")
storage_name = f"sqlite:///db.sqlite3"
# storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"

study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=50)

[I 2025-02-06 18:48:42,229] A new study created in RDB with name: 2025-02-06_18-48


A new study created in RDB with name: 2025-02-06_18-48
A new study created in RDB with name: 2025-02-06_18-48
A new study created in RDB with name: 2025-02-06_18-48
A new study created in RDB with name: 2025-02-06_18-48
A new study created in RDB with name: 2025-02-06_18-48
A new study created in RDB with name: 2025-02-06_18-48
A new study created in RDB with name: 2025-02-06_18-48
A new study created in RDB with name: 2025-02-06_18-48


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
[I 2025-02-06 18:49:59,981] Trial 0 finished with value: 38.8618140106832 and parameters: {'colsample_bytree': 0.3076890315525387, 'learning_rate': 0.037906550523780676, 'max_bin': 68, 'min_child_samples': 64, 'n_estimators': 13795, 'num_leaves': 1065, 'reg_alpha': 0.7525889942243095, 'reg_lambda': 3.66920

Trial 0 finished with value: 38.8618140106832 and parameters: {'colsample_bytree': 0.3076890315525387, 'learning_rate': 0.037906550523780676, 'max_bin': 68, 'min_child_samples': 64, 'n_estimators': 13795, 'num_leaves': 1065, 'reg_alpha': 0.7525889942243095, 'reg_lambda': 3.669202165080383e-06, 'subsample': 0.24782301432445766}. Best is trial 0 with value: 38.8618140106832.
Trial 0 finished with value: 38.8618140106832 and parameters: {'colsample_bytree': 0.3076890315525387, 'learning_rate': 0.037906550523780676, 'max_bin': 68, 'min_child_samples': 64, 'n_estimators': 13795, 'num_leaves': 1065, 'reg_alpha': 0.7525889942243095, 'reg_lambda': 3.669202165080383e-06, 'subsample': 0.24782301432445766}. Best is trial 0 with value: 38.8618140106832.
Trial 0 finished with value: 38.8618140106832 and parameters: {'colsample_bytree': 0.3076890315525387, 'learning_rate': 0.037906550523780676, 'max_bin': 68, 'min_child_samples': 64, 'n_estimators': 13795, 'num_leaves': 1065, 'reg_alpha': 0.75258899

[I 2025-02-06 18:52:11,309] Trial 1 finished with value: 38.86449198445021 and parameters: {'colsample_bytree': 0.33653672720557387, 'learning_rate': 0.020797017333403188, 'max_bin': 41, 'min_child_samples': 59, 'n_estimators': 12684, 'num_leaves': 1177, 'reg_alpha': 0.6269563443648634, 'reg_lambda': 1.3329655311093293e-06, 'subsample': 0.24757076963868546}. Best is trial 0 with value: 38.8618140106832.


...

Trial 47 finished with value: 38.85852641200999 and parameters: {'colsample_bytree': 0.302829966228656, 'learning_rate': 0.046850138627281275, 'max_bin': 242, 'min_child_samples': 59, 'n_estimators': 11200, 'num_leaves': 939, 'reg_alpha': 1.2230148171242843, 'reg_lambda': 2.9328530308219802e-06, 'subsample': 0.20027158027653472}. Best is trial 45 with value: 38.85725023088483.
Trial 47 finished with value: 38.85852641200999 and parameters: {'colsample_bytree': 0.302829966228656, 'learning_rate': 0.046850138627281275, 'max_bin': 242, 'min_child_samples': 59, 'n_estimators': 11200, 'num_leaves': 939, 'reg_alpha': 1.2230148171242843, 'reg_lambda': 2.9328530308219802e-06, 'subsample': 0.20027158027653472}. Best is trial 45 with value: 38.85725023088483.
Trial 47 finished with value: 38.85852641200999 and parameters: {'colsample_bytree': 0.302829966228656, 'learning_rate': 0.046850138627281275, 'max_bin': 242, 'min_child_samples': 59, 'n_estimators': 11200, 'num_leaves': 939, 'reg_alpha': 1

[I 2025-02-06 19:08:32,211] Trial 48 finished with value: 38.85827406792398 and parameters: {'colsample_bytree': 0.2976222212818758, 'learning_rate': 0.04673218139259328, 'max_bin': 251, 'min_child_samples': 64, 'n_estimators': 10912, 'num_leaves': 971, 'reg_alpha': 1.2702519317930194, 'reg_lambda': 2.0424857064709736e-06, 'subsample': 0.20272270489772107}. Best is trial 45 with value: 38.85725023088483.


Trial 48 finished with value: 38.85827406792398 and parameters: {'colsample_bytree': 0.2976222212818758, 'learning_rate': 0.04673218139259328, 'max_bin': 251, 'min_child_samples': 64, 'n_estimators': 10912, 'num_leaves': 971, 'reg_alpha': 1.2702519317930194, 'reg_lambda': 2.0424857064709736e-06, 'subsample': 0.20272270489772107}. Best is trial 45 with value: 38.85725023088483.
Trial 48 finished with value: 38.85827406792398 and parameters: {'colsample_bytree': 0.2976222212818758, 'learning_rate': 0.04673218139259328, 'max_bin': 251, 'min_child_samples': 64, 'n_estimators': 10912, 'num_leaves': 971, 'reg_alpha': 1.2702519317930194, 'reg_lambda': 2.0424857064709736e-06, 'subsample': 0.20272270489772107}. Best is trial 45 with value: 38.85725023088483.
Trial 48 finished with value: 38.85827406792398 and parameters: {'colsample_bytree': 0.2976222212818758, 'learning_rate': 0.04673218139259328, 'max_bin': 251, 'min_child_samples': 64, 'n_estimators': 10912, 'num_leaves': 971, 'reg_alpha': 1

[I 2025-02-06 19:09:19,169] Trial 49 finished with value: 38.85691610847639 and parameters: {'colsample_bytree': 0.3038806818979098, 'learning_rate': 0.04910535343913768, 'max_bin': 248, 'min_child_samples': 54, 'n_estimators': 12706, 'num_leaves': 969, 'reg_alpha': 1.0950209614784265, 'reg_lambda': 3.3987112639724684e-06, 'subsample': 0.20218589684713104}. Best is trial 49 with value: 38.85691610847639.


Trial 49 finished with value: 38.85691610847639 and parameters: {'colsample_bytree': 0.3038806818979098, 'learning_rate': 0.04910535343913768, 'max_bin': 248, 'min_child_samples': 54, 'n_estimators': 12706, 'num_leaves': 969, 'reg_alpha': 1.0950209614784265, 'reg_lambda': 3.3987112639724684e-06, 'subsample': 0.20218589684713104}. Best is trial 49 with value: 38.85691610847639.
Trial 49 finished with value: 38.85691610847639 and parameters: {'colsample_bytree': 0.3038806818979098, 'learning_rate': 0.04910535343913768, 'max_bin': 248, 'min_child_samples': 54, 'n_estimators': 12706, 'num_leaves': 969, 'reg_alpha': 1.0950209614784265, 'reg_lambda': 3.3987112639724684e-06, 'subsample': 0.20218589684713104}. Best is trial 49 with value: 38.85691610847639.
Trial 49 finished with value: 38.85691610847639 and parameters: {'colsample_bytree': 0.3038806818979098, 'learning_rate': 0.04910535343913768, 'max_bin': 248, 'min_child_samples': 54, 'n_estimators': 12706, 'num_leaves': 969, 'reg_alpha': 1

In [ ]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, .35),
        'learning_rate': trial.suggest_loguniform('learning_rate', .01, 0.05),
        'max_bin': 15,
        # 'max_bin': trial.suggest_int('max_bin', 100000, 130000),
        'max_depth': 10,
        'min_child_samples': trial.suggest_int('min_child_samples', 40, 80),
        'min_split_gain': 0.5,
        'n_estimators': trial.suggest_int('n_estimators', 9000, 15000),
        'num_leaves': trial.suggest_int('num_leaves', 64, 256),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.5, 2.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.000001, 0.00001),
        'subsample': trial.suggest_float('subsample', 0.2, 0.25),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
        'device': 'gpu',
        'gpu_use_dp': False,
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(
        params=param,
        train_set=train_data,
        num_boost_round=700,
        valid_sets=[train_data, valid_data],
        valid_names=['train_0', 'valid_0'],
        callbacks=[
            LightGBMPruningCallback(trial, "rmse", valid_name="valid_0"),
            lgb.log_evaluation(-1)                   # Suppress training logs
        ]
    )
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M")
storage_name = f"sqlite:///db.sqlite3"
# storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"

study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=50)

[I 2025-02-06 19:24:16,122] A new study created in RDB with name: 2025-02-06_19-24


A new study created in RDB with name: 2025-02-06_19-24
A new study created in RDB with name: 2025-02-06_19-24
A new study created in RDB with name: 2025-02-06_19-24
A new study created in RDB with name: 2025-02-06_19-24
A new study created in RDB with name: 2025-02-06_19-24
A new study created in RDB with name: 2025-02-06_19-24
A new study created in RDB with name: 2025-02-06_19-24
A new study created in RDB with name: 2025-02-06_19-24
A new study created in RDB with name: 2025-02-06_19-24
A new study created in RDB with name: 2025-02-06_19-24


[I 2025-02-06 19:25:38,879] Trial 0 finished with value: 38.870183284791956 and parameters: {'colsample_bytree': 0.28108894492242603, 'learning_rate': 0.03455068884319494, 'min_child_samples': 52, 'n_estimators': 9765, 'num_leaves': 210, 'reg_alpha': 1.0444621954128397, 'reg_lambda': 1.1007173111635054e-06, 'subsample': 0.2432732155329544}. Best is trial 0 with value: 38.870183284791956.


Trial 0 finished with value: 38.870183284791956 and parameters: {'colsample_bytree': 0.28108894492242603, 'learning_rate': 0.03455068884319494, 'min_child_samples': 52, 'n_estimators': 9765, 'num_leaves': 210, 'reg_alpha': 1.0444621954128397, 'reg_lambda': 1.1007173111635054e-06, 'subsample': 0.2432732155329544}. Best is trial 0 with value: 38.870183284791956.
Trial 0 finished with value: 38.870183284791956 and parameters: {'colsample_bytree': 0.28108894492242603, 'learning_rate': 0.03455068884319494, 'min_child_samples': 52, 'n_estimators': 9765, 'num_leaves': 210, 'reg_alpha': 1.0444621954128397, 'reg_lambda': 1.1007173111635054e-06, 'subsample': 0.2432732155329544}. Best is trial 0 with value: 38.870183284791956.
Trial 0 finished with value: 38.870183284791956 and parameters: {'colsample_bytree': 0.28108894492242603, 'learning_rate': 0.03455068884319494, 'min_child_samples': 52, 'n_estimators': 9765, 'num_leaves': 210, 'reg_alpha': 1.0444621954128397, 'reg_lambda': 1.100717311163505

[I 2025-02-06 19:29:56,943] Trial 1 finished with value: 38.872584345442554 and parameters: {'colsample_bytree': 0.2717989756008757, 'learning_rate': 0.014081584932282974, 'min_child_samples': 54, 'n_estimators': 10326, 'num_leaves': 93, 'reg_alpha': 0.8423340985618515, 'reg_lambda': 1.2099457162165117e-06, 'subsample': 0.2425554989765521}. Best is trial 0 with value: 38.870183284791956.


...

Trial 47 pruned. Trial was pruned at iteration 185.
Trial 47 pruned. Trial was pruned at iteration 185.
Trial 47 pruned. Trial was pruned at iteration 185.
Trial 47 pruned. Trial was pruned at iteration 185.
Trial 47 pruned. Trial was pruned at iteration 185.
Trial 47 pruned. Trial was pruned at iteration 185.
Trial 47 pruned. Trial was pruned at iteration 185.
Trial 47 pruned. Trial was pruned at iteration 185.
Trial 47 pruned. Trial was pruned at iteration 185.
Trial 47 pruned. Trial was pruned at iteration 185.


[I 2025-02-06 20:07:50,057] Trial 48 pruned. Trial was pruned at iteration 0.


Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.
Trial 48 pruned. Trial was pruned at iteration 0.


[I 2025-02-06 20:07:50,964] Trial 49 pruned. Trial was pruned at iteration 0.


Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.
Trial 49 pruned. Trial was pruned at iteration 0.


In [17]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, .35),
        'learning_rate': trial.suggest_loguniform('learning_rate', .01, 0.05),
        'max_bin': 15,
        # 'max_bin': trial.suggest_int('max_bin', 100000, 130000),
        'max_depth': 10,
        'min_child_samples': trial.suggest_int('min_child_samples', 40, 80),
        'min_split_gain': 0.5,
        'n_estimators': trial.suggest_int('n_estimators', 9000, 15000),
        'num_leaves': trial.suggest_int('num_leaves', 64, 256),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.5, 2.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.000001, 0.00001),
        'subsample': trial.suggest_float('subsample', 0.2, 0.25),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
        'device': 'gpu',
        'gpu_use_dp': False,
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(
        params=param,
        train_set=train_data,
        num_boost_round=10,
        valid_sets=[train_data, valid_data],
        valid_names=['train_0', 'valid_0'],
        callbacks=[
            LightGBMPruningCallback(trial, "rmse", valid_name="valid_0"),
            lgb.log_evaluation(-1)                   # Suppress training logs
        ]
    )
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M")
storage_name = f"sqlite:///db.sqlite3"
# storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"

study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=10)

[I 2025-02-06 20:09:45,765] A new study created in RDB with name: 2025-02-06_20-09


A new study created in RDB with name: 2025-02-06_20-09
A new study created in RDB with name: 2025-02-06_20-09
A new study created in RDB with name: 2025-02-06_20-09
A new study created in RDB with name: 2025-02-06_20-09
A new study created in RDB with name: 2025-02-06_20-09
A new study created in RDB with name: 2025-02-06_20-09
A new study created in RDB with name: 2025-02-06_20-09
A new study created in RDB with name: 2025-02-06_20-09
A new study created in RDB with name: 2025-02-06_20-09
A new study created in RDB with name: 2025-02-06_20-09
A new study created in RDB with name: 2025-02-06_20-09


[I 2025-02-06 20:11:16,973] Trial 0 finished with value: 38.8727553909701 and parameters: {'colsample_bytree': 0.26169432550590455, 'learning_rate': 0.03836251831468291, 'min_child_samples': 56, 'n_estimators': 11536, 'num_leaves': 123, 'reg_alpha': 0.5310662766487285, 'reg_lambda': 6.414703152252702e-06, 'subsample': 0.2408422158833617}. Best is trial 0 with value: 38.8727553909701.


Trial 0 finished with value: 38.8727553909701 and parameters: {'colsample_bytree': 0.26169432550590455, 'learning_rate': 0.03836251831468291, 'min_child_samples': 56, 'n_estimators': 11536, 'num_leaves': 123, 'reg_alpha': 0.5310662766487285, 'reg_lambda': 6.414703152252702e-06, 'subsample': 0.2408422158833617}. Best is trial 0 with value: 38.8727553909701.
Trial 0 finished with value: 38.8727553909701 and parameters: {'colsample_bytree': 0.26169432550590455, 'learning_rate': 0.03836251831468291, 'min_child_samples': 56, 'n_estimators': 11536, 'num_leaves': 123, 'reg_alpha': 0.5310662766487285, 'reg_lambda': 6.414703152252702e-06, 'subsample': 0.2408422158833617}. Best is trial 0 with value: 38.8727553909701.
Trial 0 finished with value: 38.8727553909701 and parameters: {'colsample_bytree': 0.26169432550590455, 'learning_rate': 0.03836251831468291, 'min_child_samples': 56, 'n_estimators': 11536, 'num_leaves': 123, 'reg_alpha': 0.5310662766487285, 'reg_lambda': 6.414703152252702e-06, 'su

[I 2025-02-06 20:14:01,331] Trial 1 finished with value: 38.86924615364433 and parameters: {'colsample_bytree': 0.30886145514800617, 'learning_rate': 0.015023545675722234, 'min_child_samples': 42, 'n_estimators': 13823, 'num_leaves': 96, 'reg_alpha': 0.5087318871120046, 'reg_lambda': 1.96668425514132e-06, 'subsample': 0.21582668311188474}. Best is trial 1 with value: 38.86924615364433.


...

Trial 7 pruned. Trial was pruned at iteration 0.
Trial 7 pruned. Trial was pruned at iteration 0.
Trial 7 pruned. Trial was pruned at iteration 0.
Trial 7 pruned. Trial was pruned at iteration 0.
Trial 7 pruned. Trial was pruned at iteration 0.
Trial 7 pruned. Trial was pruned at iteration 0.
Trial 7 pruned. Trial was pruned at iteration 0.
Trial 7 pruned. Trial was pruned at iteration 0.
Trial 7 pruned. Trial was pruned at iteration 0.
Trial 7 pruned. Trial was pruned at iteration 0.
Trial 7 pruned. Trial was pruned at iteration 0.


[I 2025-02-06 20:21:27,612] Trial 8 pruned. Trial was pruned at iteration 0.


Trial 8 pruned. Trial was pruned at iteration 0.
Trial 8 pruned. Trial was pruned at iteration 0.
Trial 8 pruned. Trial was pruned at iteration 0.
Trial 8 pruned. Trial was pruned at iteration 0.
Trial 8 pruned. Trial was pruned at iteration 0.
Trial 8 pruned. Trial was pruned at iteration 0.
Trial 8 pruned. Trial was pruned at iteration 0.
Trial 8 pruned. Trial was pruned at iteration 0.
Trial 8 pruned. Trial was pruned at iteration 0.
Trial 8 pruned. Trial was pruned at iteration 0.
Trial 8 pruned. Trial was pruned at iteration 0.


[I 2025-02-06 20:22:46,082] Trial 9 finished with value: 38.87023390889522 and parameters: {'colsample_bytree': 0.3260186062371755, 'learning_rate': 0.028973013094997228, 'min_child_samples': 75, 'n_estimators': 14975, 'num_leaves': 249, 'reg_alpha': 1.0289905360547627, 'reg_lambda': 6.916105684519423e-06, 'subsample': 0.24262432489295294}. Best is trial 2 with value: 38.86881943273114.


Trial 9 finished with value: 38.87023390889522 and parameters: {'colsample_bytree': 0.3260186062371755, 'learning_rate': 0.028973013094997228, 'min_child_samples': 75, 'n_estimators': 14975, 'num_leaves': 249, 'reg_alpha': 1.0289905360547627, 'reg_lambda': 6.916105684519423e-06, 'subsample': 0.24262432489295294}. Best is trial 2 with value: 38.86881943273114.
Trial 9 finished with value: 38.87023390889522 and parameters: {'colsample_bytree': 0.3260186062371755, 'learning_rate': 0.028973013094997228, 'min_child_samples': 75, 'n_estimators': 14975, 'num_leaves': 249, 'reg_alpha': 1.0289905360547627, 'reg_lambda': 6.916105684519423e-06, 'subsample': 0.24262432489295294}. Best is trial 2 with value: 38.86881943273114.
Trial 9 finished with value: 38.87023390889522 and parameters: {'colsample_bytree': 0.3260186062371755, 'learning_rate': 0.028973013094997228, 'min_child_samples': 75, 'n_estimators': 14975, 'num_leaves': 249, 'reg_alpha': 1.0289905360547627, 'reg_lambda': 6.916105684519423e-

In [18]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, .35),
        'learning_rate': trial.suggest_loguniform('learning_rate', .01, 0.05),
        'max_bin': 15,
        'max_depth': 10,
        'min_child_samples': trial.suggest_int('min_child_samples', 40, 80),
        'min_split_gain': 0.5,
        'n_estimators': trial.suggest_int('n_estimators', 9000, 15000),
        'num_leaves': trial.suggest_int('num_leaves', 64, 256),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.5, 2.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.000001, 0.00001),
        'subsample': trial.suggest_float('subsample', 0.2, 0.25),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
        'device': 'gpu',
        'gpu_use_dp': False,
    }
    
    
    # Initialize the model with these parameters
    model = lgb.train(
        params=param,
        train_set=train_data,
        num_boost_round=10,
        valid_sets=[train_data, valid_data],
        valid_names=['train_0', 'valid_0'],
        callbacks=[
            LightGBMPruningCallback(trial, "rmse", valid_name="valid_0"),
            lgb.log_evaluation(-1)                   # Suppress training logs
        ]
    )
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse


train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M")
storage_name = f"sqlite:///db.sqlite3"
# storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"

study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=10)

[I 2025-02-06 20:22:46,136] A new study created in RDB with name: 2025-02-06_20-22


A new study created in RDB with name: 2025-02-06_20-22
A new study created in RDB with name: 2025-02-06_20-22
A new study created in RDB with name: 2025-02-06_20-22
A new study created in RDB with name: 2025-02-06_20-22
A new study created in RDB with name: 2025-02-06_20-22
A new study created in RDB with name: 2025-02-06_20-22
A new study created in RDB with name: 2025-02-06_20-22
A new study created in RDB with name: 2025-02-06_20-22
A new study created in RDB with name: 2025-02-06_20-22
A new study created in RDB with name: 2025-02-06_20-22
A new study created in RDB with name: 2025-02-06_20-22
A new study created in RDB with name: 2025-02-06_20-22


[I 2025-02-06 20:24:39,078] Trial 0 finished with value: 38.87256603776188 and parameters: {'colsample_bytree': 0.266866133566952, 'learning_rate': 0.034925077493470354, 'min_child_samples': 75, 'n_estimators': 12290, 'num_leaves': 118, 'reg_alpha': 1.2250027892593398, 'reg_lambda': 1.7783639122003273e-06, 'subsample': 0.2272889028255005}. Best is trial 0 with value: 38.87256603776188.


Trial 0 finished with value: 38.87256603776188 and parameters: {'colsample_bytree': 0.266866133566952, 'learning_rate': 0.034925077493470354, 'min_child_samples': 75, 'n_estimators': 12290, 'num_leaves': 118, 'reg_alpha': 1.2250027892593398, 'reg_lambda': 1.7783639122003273e-06, 'subsample': 0.2272889028255005}. Best is trial 0 with value: 38.87256603776188.
Trial 0 finished with value: 38.87256603776188 and parameters: {'colsample_bytree': 0.266866133566952, 'learning_rate': 0.034925077493470354, 'min_child_samples': 75, 'n_estimators': 12290, 'num_leaves': 118, 'reg_alpha': 1.2250027892593398, 'reg_lambda': 1.7783639122003273e-06, 'subsample': 0.2272889028255005}. Best is trial 0 with value: 38.87256603776188.
Trial 0 finished with value: 38.87256603776188 and parameters: {'colsample_bytree': 0.266866133566952, 'learning_rate': 0.034925077493470354, 'min_child_samples': 75, 'n_estimators': 12290, 'num_leaves': 118, 'reg_alpha': 1.2250027892593398, 'reg_lambda': 1.7783639122003273e-06

[LightGBM] [Fatal] Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.
You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.
[W 2025-02-06 20:24:39,172] Trial 1 failed with parameters: {'colsample_bytree': 0.30039816656843676, 'learning_rate': 0.01964064931710759, 'min_child_samples': 52, 'n_estimators': 14253, 'num_leaves': 213, 'reg_alpha': 1.4137115707339052, 'reg_lambda': 6.800999086027943e-06, 'subsample': 0.21850362046298386} because of the following error: LightGBMError('Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.\nYou need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.').
Traceback (most recent call last):
  File "/home/j/miniforge3/envs/ml/lib/python3.9/site-packages/optuna/study/_optimize.

Trial 1 failed with parameters: {'colsample_bytree': 0.30039816656843676, 'learning_rate': 0.01964064931710759, 'min_child_samples': 52, 'n_estimators': 14253, 'num_leaves': 213, 'reg_alpha': 1.4137115707339052, 'reg_lambda': 6.800999086027943e-06, 'subsample': 0.21850362046298386} because of the following error: LightGBMError('Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.\nYou need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.').
Traceback (most recent call last):
  File "/home/j/miniforge3/envs/ml/lib/python3.9/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_6082/2821189847.py", line 26, in objective
    model = lgb.train(
  File "/home/j/miniforge3/envs/ml/lib/python3.9/site-packages/lightgbm/engine.py", line 244, in train
    train_set._update_params(params)._set

[W 2025-02-06 20:24:39,179] Trial 1 failed with value None.


Trial 1 failed with value None.
Trial 1 failed with value None.
Trial 1 failed with value None.
Trial 1 failed with value None.
Trial 1 failed with value None.
Trial 1 failed with value None.
Trial 1 failed with value None.
Trial 1 failed with value None.
Trial 1 failed with value None.
Trial 1 failed with value None.
Trial 1 failed with value None.
Trial 1 failed with value None.


LightGBMError: Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.
You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.

In [ ]:
numeric_transformer = Pipeline(
    steps=[('scaler', StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[('onehot', OneHotEncoder(handle_unknown='ignore', drop=None))]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, cat_cols)
    ],
    remainder='drop'  # drop other columns not listed
)

In [14]:
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import KFold
# from sklearn.model_selection import train_test_split, cross_val_score
# import lightgbm as lgb

# def check_gpu_support():
#     data = np.random.rand(50, 2)
#     label = np.random.randint(2, size=50)
#     train_data = lgb.Dataset(data, label=label)
#     params = {'device': 'gpu'}
#     try:
#         gbm = lgb.train(params, train_set=train_data)
#         return True
#     except Exception as e:
#         return False

# check_gpu_support()